# Corporación Favorita Grocery Sales Forecasting
### Can you accurately predict sales for a large grocery chain?

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales. They currently rely on subjective forecasting methods with very little data to back them up and very little automation to execute plans. They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

In this competition, you will be predicting the unit sales for thousands of items sold at different Favorita stores located in Ecuador. The training data includes dates, store and item information, whether that item was being promoted, as well as the unit sales. Additional files include supplementary information that may be useful in building your models.

https://www.kaggle.com/c/favorita-grocery-sales-forecasting

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [23]:
from fastai.structured import *
from fastai.column_data import *

from IPython.display import HTML

import numpy as np
import pandas as pd
np.set_printoptions(threshold = 50, edgeitems=20)

#viz
import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

#others
import subprocess
from subprocess import check_output
import gc

PATH = 'data/grocery/'

In [5]:
!ls {PATH}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


## File Descriptions and Data Field Information
#### train.csv
* Training data, which includes the target unit_sales by date, store_nbr, and item_nbr and a unique id to label rows. The target unit_sales can be integer (e.g., a bag of chips) or float (e.g., 1.5 kg of cheese). Negative values of unit_sales represent returns of that particular item. The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr. Approximately 16% of the onpromotion values in this file are NaN.

NOTE: The training data does not include rows for items that had zero unit_sales for a store/date combination. There is no information as to whether or not the item was in stock for the store on the date, and teams will need to decide the best way to handle that situation. Also, there are a small number of items seen in the training data that aren't seen in the test data.

#### test.csv
* Test data, with the date, store_nbr, item_nbr combinations that are to be predicted, along with the onpromotion information.

NOTE: The test data has a small number of items that are not contained in the training data. Part of the exercise will be to predict a new item sales based on similar products..

#### sample_submission.csv
* A sample submission file in the correct format.

#### stores.csv
* Store metadata, including city, state, type, and cluster. Cluster is a grouping of similar stores.

#### items.csv
* Item metadata, including family, class, and perishable.

NOTE: Items marked as perishable have a score weight of 1.25; otherwise, the weight is 1.0.

#### transactions.csv
* The count of sales transactions for each date, store_nbr combination. Only included for the training data timeframe.

#### oil.csv
* Daily oil price. Includes values during both the train and test data timeframe. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

#### holidays_events.csv
* Holidays and Events, with metadata

NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

#### Additional Notes
Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.

A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

## Memory optimization

In [28]:
files=check_output(["ls", f'{PATH}']).decode("utf8")

#Check the number of row of each file
for file in files.split("\n"):
    path=f'{PATH}'+file
    popenobj=subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result,error= popenobj.communicate()
    print("The file :",file,"has :",result.strip().split()[0],"rows")

The file : holidays_events.csv has : b'351' rows
The file : items.csv has : b'4101' rows
The file : oil.csv has : b'1219' rows
The file : sample_submission.csv has : b'3370465' rows
The file : stores.csv has : b'55' rows
The file : test.csv has : b'3370465' rows
The file : train.csv has : b'125497041' rows
The file : transactions.csv has : b'83489' rows
The file :  has : b'0' rows


Since train.csv has 125 mil records, it is best to consider performing some data engineering before starting any analysis.

Inspired by https://www.kaggle.com/jagangupta/memory-optimization-and-eda-on-entire-dataset

In [30]:
#check memory use for the two biggest files - train and test
# mem_train = train.memory_usage(index=True).sum()
mem_test = test.memory_usage(index=True).sum()
# print("train dataset uses ",mem_train/ 1024**2," MB")
print("test dataset uses ",mem_test/ 1024**2," MB")

# checking contents in train
test.head()

train dataset uses  5744.79866028  MB
test dataset uses  106.072799683  MB


,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [8]:
table_names = ['train', 'stores', 'oil', 'transactions',
               'holidays_events', 'items', 'test']

In [9]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]
train, stores, oil, transactions, holidays_events, items, test = tables

In [15]:
for i,t in enumerate(tables): 
    print(table_names[i])
    display(t.head())

train


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


stores


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


oil


...

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


items


,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


test


,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


The following returns summarized aggregate information to each table accross each field.

In [18]:
for i,t in enumerate(tables):
    print(table_names[i])
    display(DataFrameSummary(t).summary())

train


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.25497e+08,NaN,1.25497e+08,1.25497e+08,1.25497e+08,NaN
mean,6.27485e+07,NaN,27.4646,972769,8.55487,NaN
std,3.62279e+07,NaN,16.3305,520534,23.6052,NaN
min,0,NaN,1,96995,-15372,NaN
25%,3.13743e+07,NaN,12,522383,2,NaN
50%,6.27485e+07,NaN,28,959500,4,NaN
75%,9.41228e+07,NaN,43,1.35438e+06,9,NaN
max,1.25497e+08,NaN,54,2.12711e+06,89440,NaN
counts,125497040,125497040,125497040,125497040,125497040,103839389
uniques,125497040,1684,54,4036,258474,2


stores


,store_nbr,city,state,type,cluster
count,54,NaN,NaN,NaN,54
mean,27.5,NaN,NaN,NaN,8.48148
std,15.7321,NaN,NaN,NaN,4.69339
min,1,NaN,NaN,NaN,1
25%,14.25,NaN,NaN,NaN,4
50%,27.5,NaN,NaN,NaN,8.5
75%,40.75,NaN,NaN,NaN,13
max,54,NaN,NaN,NaN,17
counts,54,54,54,54,54
uniques,54,22,16,5,17


oil


...

,date,type,locale,locale_name,description,transferred
count,350,350,350,350,350,350
unique,312,6,3,24,103,2
top,2014-06-25,Holiday,National,Ecuador,Carnaval,False
freq,4,221,174,174,10,338
counts,350,350,350,350,350,350
uniques,312,6,3,24,103,2
missing,0,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%
types,categorical,categorical,categorical,categorical,categorical,bool


items


,item_nbr,family,class,perishable
count,4100,NaN,4100,4100
mean,1.25144e+06,NaN,2169.65,0.240488
std,587687,NaN,1484.91,0.427432
min,96995,NaN,1002,0
25%,818111,NaN,1068,0
50%,1.3062e+06,NaN,2004,0
75%,1.90492e+06,NaN,2990.5,0
max,2.13424e+06,NaN,7780,1
counts,4100,4100,4100,4100
uniques,4100,33,337,2


test


,id,date,store_nbr,item_nbr,onpromotion
count,3.37046e+06,NaN,3.37046e+06,3.37046e+06,NaN
mean,1.27182e+08,NaN,27.5,1.2448e+06,NaN
std,972969,NaN,15.5858,589836,NaN
min,1.25497e+08,NaN,1,96995,NaN
25%,1.2634e+08,NaN,14,805321,NaN
50%,1.27182e+08,NaN,27.5,1.29466e+06,NaN
75%,1.28025e+08,NaN,41,1.73002e+06,NaN
max,1.28868e+08,NaN,54,2.13424e+06,NaN
counts,3370464,3370464,3370464,3370464,3370464
uniques,3370464,16,54,3901,2


## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [21]:
train, stores, oil, transactions, holidays_events, items, test = tables

In [22]:
len(train),len(test)

(125497040, 3370464)